In [65]:
import json,os
from nltk.corpus import wordnet as wn
import pandas as pd 
from nltk.corpus import stopwords 
import numpy as np
from nltk.probability import FreqDist, MLEProbDist
from numpy import array
import string
import operator
import copy
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


stop_words = set(stopwords.words('english')) 

In [66]:
def inputques():
    words = pd.DataFrame()

    #,answers,district,month,query-type,questions,state
    tempo = pd.read_csv('all_states_files.csv')
    words = pd.concat([tempo,words])

    questions = list(words['questions'])
    answers = list(words['answers'])
    district = list(words['district'])
    state = list(words['state'])
    query_type=list(words['query-type'])
    month=list(words['month'])
    
    #print words['questions']
    #q = list(questions)
    #print q[1]

    main = []
    for w in list(questions):
        w=str(w)
        main.append(w.split(' '))

    #print  main[0:10]

    new_words = []
    all_words = []
    for w,i in enumerate(main):
        temp = []
        for j in i:
            if j not in stop_words:
                temp.append(j)
                all_words.append(j)

        new_words.append(temp)
    return [district,state,answers,query_type,month,new_words,all_words]

In [67]:
def TrainingSynonymCheck(new_words,all_words):
    realwords=[]
    for i in all_words:
        if i.isalpha():
            realwords.append(i)
    
    lemmatized_words=[]
    for i in realwords:
        n=lemmatizer.lemmatize(i)
        lemmatized_words.append(str(n))   
    
    FreqDictionary=FreqDist(lemmatized_words)
    l=sorted(FreqDictionary.items(), key=operator.itemgetter(1),reverse=True)
    
    #print l
    
    unique_words=[]
    for i in l:
        unique_words.append(i[0])
        
        
    # Taking first words from all synsets
    # synDict={}
    # flag=0
    # for word in unique_words:
    #     if word.isalpha():
    #         syns=wn.synsets(word)
    #         lst=[]
    #         for s in syns:
    #             a=s.lemmas()[0].name()
    #             if a!=word and (a not in lst):
    #                 lst.append(str(a))
    #         if lst!=[]:
    #             synDict[word]=(lst)
        
        
    # Taking the first synset
    synDict={}
    flag=0
    for word in unique_words:
        if word.isalpha():
            syns=wn.synsets(word)
            lst=[word]
            if syns!=[]:
            #for s in syns:
                s=syns[0]
                a=s.lemmas()
                for i in a:
                    f=i.name()
                    if f!=word and (f not in lst):
                        lst.append(str(f))
            if lst!=[]:
                synDict[word]=(lst)
                
    synunique_words=copy.deepcopy(unique_words)         ## synunique_words : copy that contains all the unique words in the questions initially
    
    for word in unique_words:
        i = unique_words.index(word)
        if word in synDict.keys():
            for syn in synDict[word]:            
                for j in unique_words[i+1:]:
                    if syn==j:
                        idx=unique_words.index(j)
                        unique_words[idx]=word
    
    
    
    
            
    return [unique_words,synunique_words]              # unique words : changed words , synunique_words : original words unique 

In [68]:
def Difference(new_words,all_words):
    
    lst=TrainingSynonymCheck(new_words,all_words)
    
    unique_words=lst[0]
    synunique_words=lst[1]
    
    diff={}
    for i in range(0,len(unique_words)):
        if unique_words[i]!=synunique_words[i]:
            diff[str(synunique_words[i])]=unique_words[i]
    
    #print diff
    return diff

In [69]:
def changingQuestions(new_words,all_words):
    
    diff=Difference(new_words,all_words)
        
    changed_ques=copy.deepcopy(new_words)
    
    print diff
    changed_words=np.save("saveChangedWords.npy",diff.values())
    old_words=np.save("saveOldWords.npy",diff.keys())
    
    changedindexes=[]
    for ques in changed_ques:
        for word in ques:
            if word in diff.keys():
                i=changed_ques.index(ques)
                changedindexes.append(i)
                idx=ques.index(word)
                ques[idx]=diff[word]
            
    return changed_ques

In [70]:
# def createcsv():
#     [district,state,answers,new_words,all_words]=inputques()
#     #[unique_words,synunique_words]=TrainingSynonymCheck(new_words,all_words)
#     #diff=Difference(unique_words,synunique_words)
    
#     changed_ques= changingQuestions(new_words,all_words)
#     p=len(district)
#     q=len(state)
#     r=len(answers)
#     s=len(changed_ques)
#     print p,q,r,s
#     #df1=pd.DataFrame(district,state,answers,changed_ques)
#     #print df1
#     #df1.to_csv('all_files.csv')

In [71]:
def createcsv():
    [district,state,answers,query_type,month,new_words,all_words]=inputques()
    #[unique_words,synunique_words]=TrainingSynonymCheck(new_words,all_words)
    #diff=Difference(unique_words,synunique_words)
    #new_words=np.save("savenewwords.npy",K)
    changed_ques_lst= changingQuestions(new_words,all_words)
    #print changed_ques
    changed_ques=[]
    for i in changed_ques_lst:
        k=" ".join(i)
        changed_ques.append(k)
        #print k
    #print changed_ques
    
    #print changed_ques
    p=len(district)
    q=len(state)
    r=len(answers)
    s=len(changed_ques)
    t=len(month)
    u=len(query_type)
    #print p,q,r,s,t,u
    
    data={"answers":answers,"district":district,"state":state,"month":month,"query-type":query_type,"changed-questions":changed_ques}
    #df1=pd.DataFrame(district,state,month,query_type,answers,changed_ques)
    df1=pd.DataFrame(data)
    print df1
    df1.to_csv('all_files_synonyms.csv')

In [72]:
createcsv()

{'alfalfa': 'lucerne', 'feed': 'fertilize', 'activate': 'activated', 'fertiliser': 'fertilizer', 'burning': 'burn', 'supply': 'cater', 'color': 'colour', 'pot': 'mint', 'telephone': 'phone', 'yellow': 'yellowing', 'manager': 'director', 'nitrogen': 'n', 'year': 'yr', 'laboratory': 'lab', 'gm': 'gram', 'trip': 'activate', 'litre': 'liter', 'wire': 'wired', 'eye': 'center', 'wee': 'weeed', 'blossoming': 'flowering', 'make': 'weeed', 'apply': 'applied', 'plantation': 'orchard', 'pearl': 'dropping', 'wood': 'forest', 'grade': 'level', 'husbandry': 'farming', 'issue': 'effect', 'development': 'growth', 'mixture': 'variety', 'power': 'might', 'dirt': 'soil', 'food': 'nutrient', 'corn': 'maize', 'cardamon': 'cardamom', 'effect': 'effected', 'rain': 'rainfall', 'stem': 'fore', 'base': 'stem', 'link': 'related', 'ask': 'asked', 'chili': 'chilli', 'harvest': 'crop', 'chill': 'chilles', 'info': 'information', 'g': 'gram', 'induction': 'initiation', 'drop': 'droping', 'l': 'liter', 'preparation': 

In [115]:
def changeInputQues(ques):
    changed_words=np.load("saveChangedWords.npy")
    old_words=np.load("saveOldWords.npy")
    
    
    d={}
    for i in old_words:
        idx=np.where(old_words==i)
        d[i]=changed_words[idx]
    
    print d
    
    ques_lst=ques.split(" ")
    
    new_ques=[]
    print ques_lst
    for i in ques_lst:
        print i
        if i in old_words:
            print i
            idx=np.where(old_words==i)
            i=changed_words[idx]
            print i[0]
            new_ques.append(i[0])
        else:
            new_ques.append(i)
    print ques_lst
    print new_ques
    str_new_ques=" ".join(new_ques)
    return str_new_ques

In [116]:
ques="feed of onion"

In [117]:
changed_ques=changeInputQues(ques)

{'alfalfa': array(['lucerne'], dtype='|S11'), 'feed': array(['fertilize'], dtype='|S11'), 'activate': array(['activated'], dtype='|S11'), 'fertiliser': array(['fertilizer'], dtype='|S11'), 'burning': array(['burn'], dtype='|S11'), 'supply': array(['cater'], dtype='|S11'), 'color': array(['colour'], dtype='|S11'), 'pot': array(['mint'], dtype='|S11'), 'telephone': array(['phone'], dtype='|S11'), 'yellow': array(['yellowing'], dtype='|S11'), 'manager': array(['director'], dtype='|S11'), 'nitrogen': array(['n'], dtype='|S11'), 'year': array(['yr'], dtype='|S11'), 'laboratory': array(['lab'], dtype='|S11'), 'gm': array(['gram'], dtype='|S11'), 'trip': array(['activate'], dtype='|S11'), 'litre': array(['liter'], dtype='|S11'), 'wire': array(['wired'], dtype='|S11'), 'eye': array(['center'], dtype='|S11'), 'wee': array(['weeed'], dtype='|S11'), 'collect': array(['collected'], dtype='|S11'), 'make': array(['weeed'], dtype='|S11'), 'point': array(['pointed'], dtype='|S11'), 'plantation': array

In [118]:
changed_ques

'fertilize of onion'